This is a minimal example illustrating what `saltax` can give you. 

Lanqing & Dacheng, Jun 29 2025

There will be 3 datasets in the end:
- `data`: Exactly the same as offline real data.
- `simulation`: Events reconstructed using the simulation instruction only, there is nothing else in the reconstruction process.
- `sprinkled`: Events reconstructed by mixing simulation and data. Some time it is also called `salt` and they mean the same thing. 

In [ ]:
import strax
import straxen
import saltax
import matplotlib.pyplot as plt

straxen.print_versions(("strax", "straxen", "cutax", "saltax"))

Now let's define contexts in the `saltax` fasion.

Once you define the contexts below, it will
- Try to fetch simulation instruction specified by the context
- If no instruction found, generate simulation instruction.
- Register the `saltax` plugins as well as the `cutax` and `straxen` standard ones if not replaced.

In [ ]:
runid = "037119"

In [ ]:
xedocs_version = "global_v14"

In [ ]:
# You only need runid in context when you need to compute raw_records_simu
# salt mode: reconstruction from a mixture of data and simulation
st_salt = saltax.contexts.sxenonnt(
    runid=runid,
    corrections_version=xedocs_version,
    saltax_mode="salt",
)
# simu mode: reconstruction from simulation only
st_simu = saltax.contexts.sxenonnt(
    runid=runid,
    corrections_version=xedocs_version,
    saltax_mode="simu",
)

By default, the context above will simuilate flat beta ER band at 50 Hz.

In [ ]:
??saltax.contexts.sxenonnt

In [ ]:
# Just to bind the storage so we have access to the raw_records of a small run
st_salt.storage.append(strax.DataDirectory("/project2/lgrandi/tutorial_data", readonly=True))
st_simu.storage.append(strax.DataDirectory("/project2/lgrandi/tutorial_data", readonly=True))

You can take a look that some plugins are replaced while some are not.

In [ ]:
st_simu._plugin_class_registry["peaklets"]

In [ ]:
st_simu._plugin_class_registry["microphysics_summary"]

In [ ]:
st_simu._plugin_class_registry["event_info"]

In [ ]:
st_simu._plugin_class_registry["cuts_basic"]

Now let's make some data! Note that both contexts have the same hashes until `peaklets`, where the merging happens.

In [ ]:
st_salt.key_for(runid, "peaklets")

In [ ]:
st_simu.key_for(runid, "peaklets")

In [ ]:
st_salt.key_for(runid, "records")

In [ ]:
st_simu.key_for(runid, "records")

In [ ]:
dtypes = [
    "microphysics_summary",
    "raw_records_simu",
    "records",
    "peaklets",
    "peak_basics",
    "events",
    "event_basics",
    "event_info",
]
for dt in dtypes:
    st_salt.make(runid, dt, save=dt)
for dt in dtypes:
    st_simu.make(runid, dt, save=dt)

Let's take a quick look.

In [ ]:
events_simu = st_simu.get_array(runid, "event_info")
events_salt = st_salt.get_array(runid, "event_info")

In [ ]:
plt.figure(dpi=150)
plt.scatter(events_salt["cs1"], events_salt["cs2"], alpha=0.5, label="Sprinkled Dataset")
plt.scatter(events_simu["cs1"], events_simu["cs2"], alpha=0.5, label="Simulated Dataset")
plt.legend()
plt.xlim(0, 100)
plt.ylim(0, 6000)
plt.xlabel("cS1 [PE]")
plt.ylabel("cS2 [PE]")

In an ideal worlad without ambience interference, all the orange dots will be fully overlapped with a blue dot. However, it seems not from the plot. You now starts to see what is ambience interference. See [here](https://xe1t-wiki.lngs.infn.it/doku.php?id=lanqing:ambience_interference_and_sprinkling) for details.